# Definición de detectores (tallies)

Hasta aquí las simulaciones realizadas no generaron valores de flujo, sólo estadísticas generales como el factor de multiplicación o la probabilidad de fuga. Para calcular flujos o ritmos de reacción es necesario definir detectores o *tallies*.

Los tallies cuentan una determinada la ocurrencia de determinados estimadores (colisiones, tracklength) dentro de una determinada región del espacio de fase y eventualmente con una función de peso. En la nomenclatura de OpenMC la región del espacio de fase se determina en base a *filtros* y la función de peso se denomina *score*.

## Filtros

Los filtros son utilizados para determinar la región del espacio de fase donde se contarán los eventos del tally. Existen muchos filtros (la lista completa [está en el manual](https://openmc.readthedocs.io/en/stable/pythonapi/base.html#pythonapi-tallies)). Los principales son:

* `CellFilter()`: cuenta los eventos que ocurren en una determinada celda.
* `EnergyFilter()`: cuenta los eventos en un determinado rango de energía.
* `MeshFilter()`: cuenta eventos que ocurren en una determinada grilla cartesiana.

## Scores

El score define qué se va a contar en cada evento. Algunos de los más importantes son:
* `flux`: flujo escalar.
* `current`: corriente en una superficie o en una malla.
* `absorption`: ritmo de absorciones (flujo pesado por la sección eficaz de absorción).
* `elastic`: ritmo de scattering.
* `fission`: ritmo de fisiones.
* `total`: ritmo de reacción total.
* `(n,gamma)`: ritmo de captura radiativa

## Definición de tallies

Los tallies se crean en el API de Python llamando a la función `Tally`. Los filtros se agregan modificando el atributo `.filters` y los scores en el atributo `.scores`. Si queremos que los scores de ritmos de reacción se apliquen sólo a ciertos nucleidos, podemos indicarlo con el atributo `.nuclides`.

Se puede crear más de un tally, y todos los tallies del problema deben agruparse en un objeto `Tallies()` que exporta el input en xml.

Por ejemplo, si queremos determinar el espectro en energía del flujo promediado en la esfera con una fuente puntual de 1 eV en el origen que analizamos previamente, hacemos:

In [6]:
import openmc
import numpy as np

h2o = openmc.Material()
h2o.add_nuclide("H1", 2.0, "ao")
h2o.add_nuclide("O16", 1.0, "ao")
h2o.set_density("g/cm3", 1)

mats = openmc.Materials([h2o])
mats.cross_sections = "/home/agusgimenez/KD_OPMC/Secciones/endfb-vii.1-hdf5/cross_sections.xml"
mats.export_to_xml()

esfera = openmc.    (R=5, boundary_type="vacuum")
celda = openmc.Cell(region=-esfera, fill=h2o)
universo = openmc.Universe(cells=[celda])

geom = openmc.Geometry(universo)
geom.export_to_xml()

dist_espacial = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))
dist_angular = openmc.stats.Isotropic()
dist_energetica = openmc.stats.Discrete([2.0], [1.0])

fuente = openmc.Source()
fuente.space = dist_espacial
fuente.angle = dist_angular
fuente.energy = dist_energetica

parametros = openmc.Settings()
parametros.source = fuente
parametros.run_mode = "fixed source"
parametros.batches = 100
parametros.particles = 1000
parametros.export_to_xml()

In [7]:
# Creo el tally
tally_flujo = openmc.Tally(name="espectro")

# Defino que va a contar flujo
tally_flujo.scores = ["flux"]

# Defino un filtro para que cuente eventos dentro de la esfera 
# (en este caso particular no sería necesario porque es la única celda)
filtro_celdas = openmc.filter.CellFilter(celda.id)

# Defino un filtro de energía para calcular el espectro
limites_grupo = np.logspace(np.log10(1e-5), np.log10(10), 1000)
filtro_energia = openmc.filter.EnergyFilter(limites_grupo)

# Aplico los filtros al tally
tally_flujo.filters = [filtro_celdas, filtro_energia]

# Creo un objeto Tallies
tallies = openmc.Tallies([tally_flujo])
tallies.export_to_xml()

In [8]:
!rm statepoint.*.h5
!rm summary.h5
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [13]:


estado = openmc.StatePoint("statepoint.100.h5")

tally = estado.get_tally(name="espectro")

mamamia = tally.get_pandas_dataframe()
mamamia
# filtro = tally.find_filter(openmc.EnergyFilter)
# flujo = tally.mean.reshape(np.shape(filtro.bins[1:]))
# a = loglog(filtro.bins[1:], flujo)
# a = xlabel("Energia [eV]")
# a = ylabel("Espectro")

,cell,energy low [eV],energy high [eV],nuclide,score,mean,std. dev.
0,3,0.000010,0.000010,total,flux,0.0,0.0
1,3,0.000010,0.000010,total,flux,0.0,0.0
2,3,0.000010,0.000010,total,flux,0.0,0.0
3,3,0.000010,0.000011,total,flux,0.0,0.0
4,3,0.000011,0.000011,total,flux,0.0,0.0
...,...,...,...,...,...,...,...
994,3,9.331898,9.461848,total,flux,0.0,0.0
995,3,9.461848,9.593608,total,flux,0.0,0.0
996,3,9.593608,9.727203,total,flux,0.0,0.0
997,3,9.727203,9.862658,total,flux,0.0,0.0
